<a href="https://colab.research.google.com/github/nagaokayama/captioning_scripts/blob/main/Text_cordinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. google driveのファイルを使用する場合はまずこのスクリプトを実行してください {display-mode: "form"}
# @markdown ←●に▶が入ってるマークが「セルを実行ボタン」です。これを押して実行\
# @markdown グーグルドライブの中身はファイルブラウザのgdrive/MyDriveに共有されておりここから閲覧できます


# @markdown 左端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown 目的のフォルダにマウスオーバーすると表示される縦に3点が並んだマークをクリック→「パスをコピー」でファイルパスをコピーできます
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
import csv
import re
# 共通キャプションを持ったcsvファイルを生成
# UIはたいぷスクリプトで書いてるみたい
# @title 画像名と共通キャプションを各行に書き込んだcsvを生成　{display-mode: "form"}
# @markdown ステップ1:画像ファイルのフォルダを指定
image_folder_path = ''  # @param {type:"string"}

def extract_fileName(imagesFolder):
  # 上のセルから関数ごとコピペしただけだからいつかいい感じに関数を使いまわせる形にしたい
  # globで簡単に全部のファイルをリストした後にre.searchで画像ファイルだけを抽出する

  pattern = re.compile(".*\.(png|jpg|jpeg|gif|webp|tiff)$",re.IGNORECASE)
  tmpFiles = glob.glob(f"{imagesFolder}/*.*")
  tmpFiles.sort()
  fileNameList = []

  for name in tmpFiles:
    # 画像ファイル以外の拡張子を持つファイルを除外
    basename = os.path.basename(name)
    # print(basename)
    ismatch = pattern.search(basename)
    if ismatch:
      tmp = ismatch.group(0).split(".")[0] # 拡張子を削除
      fileNameList.append(tmp)

  return fileNameList


# @markdown ステップ2: すべての画像に書き込むキャプションをカンマor読点区切りで入力\
# @markdown 共通キャプションがない場合は空欄にしてください
# unicodeでそのまま書かれちゃうけどrを入れてなければちゃんと解釈されるっぽい？
common_caption = '' # @param {type:"string"}

# 共通のキャプションを単語ごとに分割


def generate_csv_caption(image_files,common_captions=""):
    common_caption_words = re.split(r',|、', common_captions)
    data = []
    for image_file in image_files:
        # 画像の一覧を取得して共通キャプションとまとめる
        image_path = image_file.split(".")[0]
        data.append([image_path] + common_caption_words)
    data.sort()
    # print(data)

    # @markdown ステップ3: CSVファイルの書き出し先を指定
    # CSVファイルにデータを書き込み
    csv_file_path = '/content '  # @param {type:"string"}
    csv_file_path = os.path.join(csv_file_path, 'imageAndCaptions_data.csv')
    # ↑CSVファイルを保存するパスに変更してください
    # @markdown 必要事項が書けたらセルを実行
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # データを書き込む
        csv_writer.writerows(data)

    print(f"CSVファイルが {csv_file_path} に生成されました。")
    return csv_file_path

imageList = extract_fileName(image_folder_path)
generate_csv_caption(imageList,common_caption)



In [ ]:
import csv
import os
import chardet

# @title csvファイルから各画像用キャプションテキストファイルを生成　{display-mode: "form"}
# @markdown ステップ1:  csvファイルを指定
csv_file_path = '/content/imageAndCaptions_data.csv' # @param {type:"string"}
# csv_file_path = '/content/Untitled Folder/image_data.csv' # param type{"string"}
# @markdown ステップ2: キャプションファイルの書き出しフォルダを指定
text_folder_path = '/content'  # @param {type:"string"}
# @markdown 「セルを実行」ボタンを押してキャプションテキストファイルを生成

def detect_encoding(file_path):
    # エンコードの自動判定 100％の精度ではないらしい

    with open(file_path, 'rb') as file:
        rawdata = file.read()
    result = chardet.detect(rawdata)
    return result['encoding']

def getFilenamesAndTagsFromCsv(csv_file_path=csv_file_path):

    encoding = detect_encoding(csv_file_path)
    print(f"The detected encoding is: {encoding}")

    with open(csv_file_path, 'r', newline='', encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file)
        captionsList = []
        imagenameList = []
        # rowは縦だっけ？
        for row in csv_reader:
            # 各行の要素について処理
            imagenameList.append(row[0])
            # row[0]以外のすべての要素をタグとして配列に入れる
            # 空の要素をテキストファイルに入れないように排除
            tmp = [i for i in row[1:] if i != ""]
            captionsList.append(tmp)

    return imagenameList, captionsList


def generateTextFileFromList(imagenameList, captionsList):
    for n, imagename in enumerate(imagenameList):
        # captionsのためにわざわざenumerate()つけるのなんかやだかも
        captions = captionsList[n]
        text_file_path = f'{text_folder_path}/{imagename}.txt'

        concat_captions = "、".join(captions)
        with open(text_file_path, 'w') as text_file:
            text_file.write(concat_captions)

        print(f"キャプションファイル {imagename} を {text_file_path}に保存済み")

imagenameList, captionsList = getFilenamesAndTagsFromCsv(csv_file_path)
generateTextFileFromList(imagenameList, captionsList)

In [ ]:
import re
import os
import glob

# @title 各画像ファイル名をテキストファイルに書き込む {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

# @markdown 左端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown  目的のフォルダにマウスオーバーすると表示される縦に3点が並んだマークをクリック→「パスをコピー」でファイルパスをコピーできます

# @markdown 画像の入っているフォルダを指定
image_folder = "/content/Untitled Folder" #@param {type:"string"}
# @markdown 「セルを実行」ボタンを押してテキストファイルを生成\
# @markdown テキストファイルはimage_folderに指定したフォルダに作成されます。


#フォルダとテキストのファイル名は自動で決める

def extract_fileName(imagesFolder):
  # globで簡単に全部のファイルをリストした後にre.searchで画像ファイルだけを抽出する

  pattern = re.compile(".*\.(png|jpg|jpeg|gif|webp|tiff)$",re.IGNORECASE)
  tmpFiles = glob.glob(f"{imagesFolder}/*.*")
  tmpFiles.sort()
  fileNameList = []

  for name in tmpFiles:
    # 画像ファイル以外の拡張子を持つファイルを除外
    basename = os.path.basename(name)
    # print(basename)
    ismatch = pattern.search(basename)
    if ismatch:
      tmp = ismatch.group(0).split(".")[0] # 拡張子を削除
      fileNameList.append(tmp)

  return fileNameList


def export_ImageFileName(textstring):
  text = ""
  for i in textstring:
    # 配列の各要素を指定した文字列を間に挟みつつ全結合
    text = "\n".join(textstring)

  # print(text)
  filepath = os.path.join(image_folder,"imageFileName_List.txt")
  with open(filepath, mode = "w") as f:
    f.write(text)
  return f"テキストファイルを{filepath}に生成"

# 関数を実行
tmp = extract_fileName(image_folder)
export_ImageFileName(tmp)

In [ ]:
# @markdown 開発者用セル　{display-mode: "form"}
import zipfile
import shutil
file = "/content/test"
shutil.make_archive("/content/compress", "zip",file)